In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV
import numpy as np

In [0]:
def processData(data):
  copy = data.drop(['Sex'], axis=1)
  copy.fillna(0, inplace=True) #Replacing all missing data with 0
  copy['Cabin'][copy['Cabin'] != 0] = 1
  xEncoder = preprocessing.OneHotEncoder(handle_unknown='ignore')
  x1 = data[['Sex']]
  xEncoder.fit(x1)
  x1 = xEncoder.transform(x1).toarray()
  df = pd.DataFrame(data=x1, columns=['Male', 'Female'])
  output = pd.concat([copy, df], axis=1)
  output=(output-output.min())/(output.max()-output.min()) #MinMax normalization
  #print('After normalization')
  #print(output.head())
  return output

In [0]:
df = pd.read_csv('/content/drive/My Drive/Dataset/Titanic/train.csv', header=0)
train, test = train_test_split(df, test_size=0.2)
train.reset_index(inplace=True, drop=True)
test.reset_index(inplace=True, drop=True)
trainX = train.drop(['Survived', 'PassengerId', 'Name', 'Ticket', 'Embarked'], axis=1)
testX = test.drop(['Survived', 'PassengerId', 'Name', 'Ticket', 'Embarked'], axis=1)
trainY = train[['Survived']]
testY = test[['Survived']]
trainX = processData(trainX)
testX = processData(testX)

In [31]:
singleNN = MLPClassifier(max_iter=500, solver='sgd',learning_rate='adaptive', random_state=42, learning_rate_init=1.5, batch_size=600, hidden_layer_sizes=5000, alpha=0.00005)
singleNN.fit(trainX, trainY.values.ravel())
print('k')
print(singleNN.score(testX, testY))
print('k')
singleNN.score(trainX, trainY)

k
0.8044692737430168
k


0.8441011235955056

In [0]:
hyperPara = {'solver':['adam', 'sgd'], 'learning_rate':['adaptive', 'constant'], 'learning_rate_init':np.linspace(0.8, 2, 7), 'shuffle':[True, False]}
gsNN = GridSearchCV(MLPClassifier(solver='sgd', max_iter=500, random_state=42, hidden_layer_sizes=5000, batch_size=600), hyperPara)
gsNN.fit(trainX, trainY.values.ravel())

In [25]:
print(gsNN.best_params_)
gsNN.score(testX, testY)

{'learning_rate': 'constant', 'learning_rate_init': 1.7999999999999998, 'shuffle': True, 'solver': 'sgd'}


0.7821229050279329

In [0]:
hyperPara2 = {'learning_rate_init':np.linspace(0.8, 2, 7), 'shuffle':[True, False], 'momentum':[0.9, 0.6, 0.3], 'alpha':[0.00005, 0.0001]}
sgdNN = GridSearchCV(MLPClassifier(solver='sgd', learning_rate='adaptive', max_iter=500, random_state=42, hidden_layer_sizes=1000, batch_size=400), hyperPara2)
sgdNN.fit(trainX, trainY.values.ravel())

In [30]:
print(sgdNN.best_params_)
sgdNN.score(testX, testY)

{'alpha': 5e-05, 'learning_rate_init': 1.0, 'momentum': 0.6, 'shuffle': True}


0.7932960893854749